In [1]:
import cv2
import time

# Define a cor das caixas delimitadoras
colors = ('red', 'green', 'blue', '')
class_names = []

with open("coco.names", "r") as f:
    class_names = [cname.strip() for cname in f.readlines()]

# Captura de vídeo
cap = cv2.VideoCapture("avenida.mp4")

# Carregando pesos da rede neural
net = cv2.dnn.readNet("yolov4.weights", "yolov4.cfg")

# Configurar para uso da GPU
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)

# Setando parâmetros
model = cv2.dnn.DetectionModel(net)
model.setInputParams(size=(416, 416), scale=1/255)

carCounter = 0
line_position = 500  # Define a posição da linha horizontal (ajuste conforme necessário)

# Função para verificar se uma caixa cruzou a linha
def crossed_the_line(box, line_position):
    x, y, w, h = box
    return y < line_position < y + h

# Lendo frames do vídeo
while True:
    start_time = time.time()
    ret, frame = cap.read()
    if not ret:
        break

    classes, scores, boxes = model.detect(frame, 0.01, 0.2)
    
    for (class_id, score, box) in zip(classes, scores, boxes):
        try:
            label = f"{class_names[class_id]}: {score:.2f}"
        except:
            print(0)
        if class_id == 0:  # ID da classe para 'carro'

            carCounter += 1

        # Desenhar caixas delimitadoras
        color = (255, 10, 10)
        cv2.rectangle(frame, box, (255, 10, 10), 2)
        cv2.putText(frame, label, (box[0], box[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)

    # Desenhar a linha de contagem
    cv2.line(frame, (0, line_position), (frame.shape[1], line_position), (0, 255, 255), 2)
    
    # Mostrar contagem de carros
    cv2.putText(frame, f"Carros: {carCounter}", (130, 20), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    end_time = time.time()
    fps_label = f"Tempo do frame: {(end_time - start_time):.2f}s"
    cv2.putText(frame, fps_label, (130, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
    #cv2.rectangle(frame, (100,100),(540,0), (0, 0, 0), -1)
    print(carCounter)
    carCounter=0

    cv2.imshow('frame', frame)

    # Condição de término do loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Liberando os recursos
cap.release()
cv2.destroyAllWindows()


1
1
2
2
1
1
1


KeyboardInterrupt: 

In [6]:
import cv2
count = cv2.cuda.getCudaEnabledDeviceCount()
print(count)

0
